# Hard-sphere model

1D/2D/3D hard-sphere setup with validation for box bounds and overlaps (r = 1/2).

In [ ]:
from pathlib import Path
import sys
import importlib
import time

import numpy as np
import json
import plotly.graph_objects as go

cwd = Path.cwd().resolve()
root = None
for candidate in [cwd, *cwd.parents]:
    if (candidate / "hard_sphere.py").exists():
        root = candidate
        break
if root is None:
    raise RuntimeError("Could not locate hard_sphere.py")
if str(root) not in sys.path:
    sys.path.insert(0, str(root))

import hard_spheres
importlib.reload(hard_spheres)
from hard_spheres import HardSpheresSimulation, validate_initial_conditions


## Beta test outline

- overlap-2d: L=10, N=1..20, trials=1000, seed=None
- overlap-3d: L=10, N=1..60, trials=5000, seed=None
- mean-free-path-2d: L=10, N=15, Nd=30, d=1.0, seed=None
- mean-free-path-3d: L=10, N=30, Nd=30, d=1.0, seed=None
- validation-compare-2d: L=10, N=7, trials=10000, seed=None
- validation-compare-3d: L=10, N=17, trials=10000, seed=None
- validation-timing-2d: L=10, N=7, moves=10000, dl=1.0, seed=12345
- validation-timing-3d: L=10, N=17, moves=10000, dl=1.0, seed=12345

Results are cached under `beta-tests/<name>/results.npz`.

In [ ]:
beta_tests = [
    {
        "name": "overlap-2d",
        "dim": 2,
        "Lbox": 10.0,
        "r": 0.5,
        "n_min": 1,
        "n_max": 20,
        "n_trials": 1000,
        "seed": None,
    },
    {
        "name": "overlap-3d",
        "dim": 3,
        "Lbox": 10.0,
        "r": 0.5,
        "n_min": 1,
        "n_max": 60,
        "n_trials": 5000,
        "seed": None,
    },
    {
        "name": "mean-free-path-2d",
        "dim": 2,
        "Lbox": 10.0,
        "r": 0.5,
        "n_spheres": 15,
        "n_directions": 30,
        "step": 1.0,
        "seed": None,
    },
    {
        "name": "mean-free-path-3d",
        "dim": 3,
        "Lbox": 10.0,
        "r": 0.5,
        "n_spheres": 30,
        "n_directions": 30,
        "step": 1.0,
        "seed": None,
    },
    {
        "name": "validation-compare-2d",
        "dim": 2,
        "Lbox": 10.0,
        "r": 0.5,
        "n_spheres": 7,
        "n_trials": 10000,
        "seed": None,
    },
    {
        "name": "validation-compare-3d",
        "dim": 3,
        "Lbox": 10.0,
        "r": 0.5,
        "n_spheres": 17,
        "n_trials": 10000,
        "seed": None,
    },
    {
        "name": "validation-timing-2d",
        "dim": 2,
        "Lbox": 10.0,
        "r": 0.5,
        "n_spheres": 7,
        "move_trials": 10000,
        "dl": 1.0,
        "seed": 12345,
    },
    {
        "name": "validation-timing-3d",
        "dim": 3,
        "Lbox": 10.0,
        "r": 0.5,
        "n_spheres": 17,
        "move_trials": 10000,
        "dl": 1.0,
        "seed": 12345,
    },
]

output_root = root / "beta-tests"
for test in beta_tests:
    (output_root / test["name"]).mkdir(parents=True, exist_ok=True)

filename = output_root / beta_tests[0]["name"] / "results.npz"
ReadResultsFromFile = True


### overlap-2D

Beta test: random placement overlap rate for $N=1..20$ in a 2D box with $L=10$ and $r=0.5$.

In [ ]:
### Beta tests for overlap rate in 2D
def overlap_rate_by_n(
    n_trials: int,
    seed: int | None,
    n_min: int,
    n_max: int,
    Lbox: float,
    r: float,
) -> dict[int, float]:
    rng = np.random.default_rng(seed)
    limit = Lbox * 0.5 - r
    rates: dict[int, float] = {}

    for n in range(n_min, n_max + 1):
        overlaps = 0
        for _ in range(n_trials):
            coords = rng.uniform(-limit, limit, size=(n, 2))
            try:
                validate_initial_conditions(coords, 2, Lbox, r)
            except ValueError:
                overlaps += 1
        rates[n] = overlaps / n_trials

    return rates

def plot_overlap_rate_by_n(
  xs,
  ys,
  params
):
  labels = [f"{y:.1f}%" for y in ys]
  text_y = [y if y > 0.0 else 1.0 for y in ys]
  zero_xs = [x for x, y in zip(xs, ys) if y == 0.0]

  fig = go.Figure(
    data=[
      go.Bar(
        x=xs,
        y=ys,
        name=f"Trials: {params['n_trials']}",
        marker_color="steelblue",
        hovertemplate="N=%{x}<br>Overlap=%{y:.1f}%<extra></extra>",
      ),
      go.Scatter(
        x=xs,
        y=text_y,
        text=labels,
        mode="text",
        textposition="top center",
        textfont=dict(size=8),
        cliponaxis=False,
        showlegend=False,
        hoverinfo="skip",
      ),
    ]
  )

  for x in zero_xs:
      fig.add_shape(
        type="line",
        x0=x - 0.4,
        x1=x + 0.4,
        y0=0,
        y1=0,
        line=dict(color="red", width=3),
      )

  fig.update_layout(
    title=(
        "Overlap rate vs N (2D)"
        f"<br>L={params['Lbox']}, r={params['r']}"
    ),
    xaxis_title="N",
    yaxis_title="Overlap rate (%)",
    width=1000,
    height=800,
  )
  fig.update_xaxes(
    tickmode="array",
    tickvals=xs,
    range=[0.5, max(xs) + 0.5],
  )
  fig.update_yaxes(range=[0, 100])
  return fig

test = beta_tests[0]
filename = output_root / test["name"] / "results.npz"
if ReadResultsFromFile and filename.exists():
  data = np.load(filename)
  xs = data["xs"].tolist()
  ys = data["ys"].tolist()
  params = json.loads(str(data["params_json"]))
else:
  rates = overlap_rate_by_n(
    test["n_trials"],
    test["seed"],
    test["n_min"],
    test["n_max"],
    test["Lbox"],
    test["r"],
  )
  xs = list(rates.keys())
  ys = [rate * 100.0 for rate in rates.values()]
  params = test
  np.savez(
    filename,
    xs=np.array(xs),
    ys=np.array(ys),
    params_json=json.dumps(params),
  )

plot_overlap_rate_by_n(
  xs,
  ys,
  params,
).show(renderer="png")

In [ ]:
### Beta tests for overlap rate in 2D - one-off test
try:
  raise Exception("Comment me to do a one-off test")
  test = beta_tests[0]
  rates = overlap_rate_by_n(
    test["n_trials"],
    test["seed"],
    test["n_min"],
    test["n_max"],
    test["Lbox"],
    test["r"],
  )
  xs = list(rates.keys())
  ys = [rate * 100.0 for rate in rates.values()]

  plot_overlap_rate_by_n(
    xs,
    ys,
    test,
  ).show()
except Exception as e:
  print(e)


### overlap-3d

Beta test: random placement overlap rate for $N=1..60$ in a 3D box with $L=10$ and $r=0.5$.

In [ ]:
# Beta tests for overlap rate in 3D
def overlap_rate_by_n_3d(
  n_trials: int,
  seed: int | None,
  n_min: int,
  n_max: int,
  Lbox: float,
  r: float,
) -> dict[int, float]:
  rng = np.random.default_rng(seed)
  limit = Lbox * 0.5 - r
  rates: dict[int, float] = {}

  for n in range(n_min, n_max + 1):
    overlaps = 0
    for _ in range(n_trials):
      coords = rng.uniform(-limit, limit, size=(n, 3))
      try:
        validate_initial_conditions(coords, 3, Lbox, r)
      except ValueError:
        overlaps += 1
    rates[n] = overlaps / n_trials

  return rates


def plot_overlap_rate_by_n_3d(
  xs_3d,
  ys_3d,
  params
):
  zero_xs_3d = [x for x, y in zip(xs_3d, ys_3d) if y == 0.0]

  fig_3d = go.Figure(
    data=[
      go.Bar(
        x=xs_3d,
        y=ys_3d,
        name=f"Trials: {params['n_trials']}",
        marker_color="steelblue",
        hovertemplate="N=%{x}<br>Overlap=%{y:.1f}%<extra></extra>",
      )
    ]
  )

  for x in zero_xs_3d:
    fig_3d.add_shape(
      type="line",
      x0=x - 0.4,
      x1=x + 0.4,
      y0=0,
      y1=0,
      line=dict(color="red", width=3),
    )

  fig_3d.update_layout(
    title=(
      "Overlap rate vs N (3D)"
      f"<br>L={params['Lbox']}, r={params['r']}"
    ),
    xaxis_title="N",
    yaxis_title="Overlap rate (%)",
    width=1000,
    height=800,
    showlegend=True,
  )
  fig_3d.update_xaxes(
    tickmode="linear",
    tick0=1,
    dtick=5,
    range=[0.5, max(xs_3d) + 0.5],
  )
  fig_3d.update_yaxes(range=[0, 100])
  return fig_3d


test = beta_tests[1]
filename = output_root / test["name"] / "results.npz"
if ReadResultsFromFile and filename.exists():
  data = np.load(filename)
  xs_3d = data["xs"].tolist()
  ys_3d = data["ys"].tolist()
  params = json.loads(str(data["params_json"]))
else:
  rates_3d = overlap_rate_by_n_3d(
    test["n_trials"],
    test["seed"],
    test["n_min"],
    test["n_max"],
    test["Lbox"],
    test["r"],
  )
  xs_3d = list(rates_3d.keys())
  ys_3d = [rate * 100.0 for rate in rates_3d.values()]
  params = test
  np.savez(
    filename,
    xs=np.array(xs_3d),
    ys=np.array(ys_3d),
    params_json=json.dumps(params),
  )

plot_overlap_rate_by_n_3d(
  xs_3d,
  ys_3d,
  params,
).show(renderer="png")

In [ ]:
# Beta tests for overlap rate in 3D - one-off test
try:
  raise Exception("Comment me to do a one-off test")
  test = beta_tests[1]
  rates_3d = overlap_rate_by_n_3d(
    test["n_trials"],
    test["seed"],
    test["n_min"],
    test["n_max"],
    test["Lbox"],
    test["r"],
  )
  xs_3d = list(rates_3d.keys())
  ys_3d = [rate * 100.0 for rate in rates_3d.values()]

  plot_overlap_rate_by_n_3d(
    xs_3d,
    ys_3d,
    test,
  ).show()
except Exception as e:
  print(e)


### mean-free-path-2d

Beta test: mean pseudo free path for a 2D box with $L=10$, $N=15$, $Nd=30$, and $d=1.0$.

In [ ]:
# Beta tests for mean free path in 2D
def compute_mean_free_path_2d(
  test
) -> tuple[np.ndarray, float]:
  rng = np.random.default_rng(test["seed"])
  limit = test["Lbox"] * 0.5 - test["r"]

  coords = None
  while coords is None:
    trial_coords = rng.uniform(
      -limit, limit, size=(test["n_spheres"], test["dim"])
)
    try:
      coords = validate_initial_conditions(
        trial_coords, test["dim"], test["Lbox"], test["r"]
      )
    except ValueError:
      coords = None

  sim = HardSpheresSimulation(dim=test["dim"], Lbox=test["Lbox"], r=test["r"])
  sim.SetInitialConditions(coords)
  mean_free_path = sim.MeanPseudoFreePath(
    n_directions=test["n_directions"], step=test["step"], seed=test["seed"]
  )

  return coords, mean_free_path


def plot_mean_free_path_2d(
  coords,
  params,
  mean_free_path
):
  theta = np.linspace(0.0, 2.0 * np.pi, 200)
  base_r = params["r"]
  shell_r = params["r"] + params["step"]
  half = params["Lbox"] * 0.5

  fig = go.Figure()
  fig.add_shape(
    type="rect",
    x0=-half,
    x1=half,
    y0=-half,
    y1=half,
    line=dict(color="gray", width=1),
  )

  for cx, cy in coords:
    fig.add_trace(
      go.Scatter(
        x=cx + base_r * np.cos(theta),
        y=cy + base_r * np.sin(theta),
        mode="lines",
        line=dict(color="black", width=2),
        showlegend=False,
        hoverinfo="skip",
      )
    )
    fig.add_trace(
      go.Scatter(
        x=cx + shell_r * np.cos(theta),
        y=cy + shell_r * np.sin(theta),
        mode="lines",
        line=dict(color="red", width=1, dash="dot"),
        showlegend=False,
        hoverinfo="skip",
      )
    )

  scale_x0 = -half + 0.5
  scale_x1 = scale_x0 + 1.0
  scale_y = -half + 0.5
  fig.add_shape(
    type="line",
    x0=scale_x0,
    x1=scale_x1,
    y0=scale_y,
    y1=scale_y,
    line=dict(color="black", width=3),
  )
  fig.add_annotation(
    x=(scale_x0 + scale_x1) * 0.5,
    y=scale_y - 0.2,
    text="1 unit",
    showarrow=False,
    font=dict(size=10, color="black"),
  )

  fig.update_layout(
    title=(
      "Mean pseudo free path setup (2D)<br>"
      f"L={params['Lbox']}, N={params['n_spheres']}, "
      f"Nd={params['n_directions']}, d={params['step']}, "
      f"mpfp={mean_free_path:.4f}"
    ),
    width=700,
    height=700,
    plot_bgcolor="white",
  )
  fig.update_xaxes(range=[-half, half], visible=False)
  fig.update_yaxes(range=[-half, half], scaleanchor="x", scaleratio=1, visible=False)
  return fig


test = beta_tests[2]
filename = output_root / test["name"] / "results.npz"
if ReadResultsFromFile and filename.exists():
  data = np.load(filename)
  coords = data["coords"]
  mean_free_path = float(data["mean_free_path"])
  params = json.loads(str(data["params_json"]))
else:
  coords, mean_free_path = compute_mean_free_path_2d(test)
  params = test
  np.savez(
    filename,
    coords=coords,
    mean_free_path=mean_free_path,
    params_json=json.dumps(params),
  )

print(
  "Mean pseudo free path "
  f"(L={params['Lbox']}, N={params['n_spheres']}, Nd={params['n_directions']}, "
  f"d={params['step']}): {mean_free_path:.4f}"
)

plot_mean_free_path_2d(
  coords,
  params,
  mean_free_path,
).show(renderer="png")


In [ ]:
# Beta tests for mean free path in 2D - one-off test
try:
  test = beta_tests[2]
  raise Exception("Comment me to do a one-off test")
  coords, mean_free_path = compute_mean_free_path_2d(test)

  plot_mean_free_path_2d(
    coords,
    test,
    mean_free_path,
  ).show()
except Exception as e:
  print(e)


### mean-free-path-3d

Beta test: mean pseudo free path for a 3D box with $L=10$, $N=30$, $Nd=30$, and $d=1.0$.

In [ ]:
# Beta tests for mean free path in 3D
def compute_mean_free_path_3d(
  test
) -> tuple[np.ndarray, float]:
  rng = np.random.default_rng(test["seed"])
  limit = test["Lbox"] * 0.5 - test["r"]

  coords = None
  while coords is None:
    trial_coords = rng.uniform(
      -limit, limit, size=(test["n_spheres"], test["dim"])
)
    try:
      coords = validate_initial_conditions(
        trial_coords, test["dim"], test["Lbox"], test["r"]
      )
    except ValueError:
      coords = None

  sim = HardSpheresSimulation(dim=test["dim"], Lbox=test["Lbox"], r=test["r"])
  sim.SetInitialConditions(coords)
  mean_free_path = sim.MeanPseudoFreePath(
    n_directions=test["n_directions"], step=test["step"], seed=test["seed"]
  )

  return coords, mean_free_path


def plot_mean_free_path_3d(
  coords,
  params,
  mean_free_path
):
  base_r = params["r"]
  shell_r = params["r"] + params["step"]
  half = params["Lbox"] * 0.5

  def sphere_surface(center: tuple[float, float, float], r: float) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    theta = np.linspace(0.0, 2.0 * np.pi, 24)
    phi = np.linspace(0.0, np.pi, 16)
    theta, phi = np.meshgrid(theta, phi)
    x = center[0] + r * np.cos(theta) * np.sin(phi)
    y = center[1] + r * np.sin(theta) * np.sin(phi)
    z = center[2] + r * np.cos(phi)
    return x, y, z

  cube = np.array([
    [-half, -half, -half],
    [ half, -half, -half],
    [ half,  half, -half],
    [-half,  half, -half],
    [-half, -half,  half],
    [ half, -half,  half],
    [ half,  half,  half],
    [-half,  half,  half],
  ])
  edges = [
    (0, 1), (1, 2), (2, 3), (3, 0),
    (4, 5), (5, 6), (6, 7), (7, 4),
    (0, 4), (1, 5), (2, 6), (3, 7),
  ]

  fig = go.Figure()
  for i, j in edges:
    fig.add_trace(
      go.Scatter3d(
        x=[cube[i, 0], cube[j, 0]],
        y=[cube[i, 1], cube[j, 1]],
        z=[cube[i, 2], cube[j, 2]],
        mode="lines",
        line=dict(color="gray", width=3),
        showlegend=False,
        hoverinfo="skip",
      )
    )

  for cx, cy, cz in coords:
    x, y, z = sphere_surface((float(cx), float(cy), float(cz)), base_r)
    fig.add_trace(
      go.Surface(
        x=x,
        y=y,
        z=z,
        showscale=False,
        colorscale=[[0, "black"], [1, "black"]],
        opacity=1.0,
      )
    )
    xs, ys, zs = sphere_surface((float(cx), float(cy), float(cz)), shell_r)
    fig.add_trace(
      go.Surface(
        x=xs,
        y=ys,
        z=zs,
        showscale=False,
        colorscale=[[0, "red"], [1, "red"]],
        opacity=0.3,
      )
    )

  fig.update_layout(
    title=(
      "Mean pseudo free path setup (3D)<br>"
      f"L={params['Lbox']}, N={params['n_spheres']}, "
      f"Nd={params['n_directions']}, d={params['step']}, "
      f"mpfp={mean_free_path:.4f}"
    ),
    width=600,
    height=600,
    margin=dict(l=0, r=0, t=50, b=0),
    scene=dict(
      xaxis=dict(visible=False, range=[-half-0.01, half+0.01]),
      yaxis=dict(visible=False, range=[-half-0.01, half+0.01]),
      zaxis=dict(visible=False, range=[-half-0.01, half+0.01]),
      aspectmode="manual",
      aspectratio=dict(x=1.5, y=1.5, z=1.5),
      camera=dict(
        projection=dict(type="orthographic"),
        eye=dict(x=2, y=1, z=0.5),
      ),
    ),
  )
  return fig


test = beta_tests[3]
filename = output_root / test["name"] / "results.npz"
if ReadResultsFromFile and filename.exists():
  data = np.load(filename)
  coords = data["coords"]
  mean_free_path = float(data["mean_free_path"])
  params = json.loads(str(data["params_json"]))
else:
  coords, mean_free_path = compute_mean_free_path_3d(test)
  params = test
  np.savez(
    filename,
    coords=coords,
    mean_free_path=mean_free_path,
    params_json=json.dumps(params),
  )

print(
  "Mean pseudo free path "
  f"(L={params['Lbox']}, N={params['n_spheres']}, Nd={params['n_directions']}, "
  f"d={params['step']}): {mean_free_path:.4f}"
)

plot_mean_free_path_3d(
  coords,
  params,
  mean_free_path,
).show(renderer="png")


In [ ]:
# Beta tests for mean free path in 3D - one-off test
try:
  raise Exception("Comment me to do a one-off test")
  test = beta_tests[2]
  rng = np.random.default_rng(test["seed"])
  limit = test["Lbox"] * 0.5 - test["r"]

  coords = None
  while coords is None:
    trial_coords = rng.uniform(
      -limit, limit, size=(test["n_spheres"], test["dim"])
)
    try:
      coords = validate_initial_conditions(
        trial_coords, test["dim"], test["Lbox"], test["r"]
      )
    except ValueError:
      coords = None

  sim = HardSphereSimulation(dim=test["dim"], Lbox=test["Lbox"], r=test["r"])
  sim.SetInitialConditions(coords)
  mean_free_path = sim.MeanPseudoFreePath(
    n_directions=test["n_directions"], step=test["step"], seed=test["seed"]
  )

  plot_mean_free_path_2d(
    coords,
    test,
    mean_free_path,
  ).show()
except Exception as e:
  print(e)


### validation-compare-2d

Beta test: compare validation vs simulation overlap detection for $N=7$ in a 2D box with $L=10$ and $r=0.5$.

In [ ]:
# Validation parity test in 2D
def compare_validation_methods(
  test
) -> dict[str, float]:
  rng = np.random.default_rng(test["seed"])
  limit = test["Lbox"] * 0.5 - test["r"]
  matches = 0
  validate_accepts = 0
  sim_accepts = 0

  for _ in range(test["n_trials"]):
    coords = rng.uniform(-limit, limit, size=(test["n_spheres"], test["dim"]))
    try:
      validate_initial_conditions(
        coords, test["dim"], test["Lbox"], test["r"]
      )
      validate_ok = True
    except ValueError:
      validate_ok = False

    sim = HardSpheresSimulation(
      dim=test["dim"], Lbox=test["Lbox"], r=test["r"]
    )
    try:
      sim.SetInitialConditions(coords)
      sim_ok = not sim.DetectOverlaps()
    except ValueError:
      sim_ok = False

    if validate_ok:
      validate_accepts += 1
    if sim_ok:
      sim_accepts += 1
    if sim_ok == validate_ok:
      matches += 1

  trials = test["n_trials"]
  return {
    "matches": matches,
    "trials": trials,
    "accuracy": matches / trials,
    "validate_accepts": validate_accepts,
    "sim_accepts": sim_accepts,
  }


test = beta_tests[4]
filename = output_root / test["name"] / "results.npz"
if ReadResultsFromFile and filename.exists():
  data = np.load(filename)
  results = json.loads(str(data["results_json"]))
  params = json.loads(str(data["params_json"]))
else:
  results = compare_validation_methods(test)
  params = test
  np.savez(
    filename,
    results_json=json.dumps(results),
    params_json=json.dumps(params),
  )

print(
  "Validation parity (2D): "
  f"accuracy={results['accuracy'] * 100.0:.2f}%, "
  f"validate accepts={results['validate_accepts']}/{results['trials']}, "
  f"sim accepts={results['sim_accepts']}/{results['trials']}"
)

### validation-compare-3d

Beta test: compare validation vs simulation overlap detection for $N=17$ in a 3D box with $L=10$ and $r=0.5$.

In [ ]:
# Validation parity test in 3D
test = beta_tests[5]
filename = output_root / test["name"] / "results.npz"
if ReadResultsFromFile and filename.exists():
  data = np.load(filename)
  results = json.loads(str(data["results_json"]))
  params = json.loads(str(data["params_json"]))
else:
  results = compare_validation_methods(test)
  params = test
  np.savez(
    filename,
    results_json=json.dumps(results),
    params_json=json.dumps(params),
  )

print(
  "Validation parity (3D): "
  f"accuracy={results['accuracy'] * 100.0:.2f}%, "
  f"validate accepts={results['validate_accepts']}/{results['trials']}, "
  f"sim accepts={results['sim_accepts']}/{results['trials']}"
)

### validation-timing-2d

Beta test: compare timing for old vs new validation method for $N=7$ in a 2D box with $L=10$ and $r=0.5$.

In [ ]:
# Validation timing test in 2D
def _random_direction(rng: np.random.Generator, dim: int) -> np.ndarray:
  if dim == 1:
    return np.array([1.0 if rng.random() < 0.5 else -1.0])
  vec = rng.normal(size=dim)
  norm = float(np.linalg.norm(vec))
  while norm == 0.0:
    vec = rng.normal(size=dim)
    norm = float(np.linalg.norm(vec))
  return vec / norm


def _find_valid_coords_old(
  rng: np.random.Generator, test, limit: float
 ) -> np.ndarray:
  while True:
    coords = rng.uniform(-limit, limit, size=(test["n_spheres"], test["dim"]))
    try:
      return validate_initial_conditions(
        coords, test["dim"], test["Lbox"], test["r"]
      )
    except ValueError:
      pass


def _find_valid_sim_new(
  rng: np.random.Generator, test, limit: float
 ) -> HardSpheresSimulation:
  while True:
    coords = rng.uniform(-limit, limit, size=(test["n_spheres"], test["dim"]))
    sim = HardSpheresSimulation(
      dim=test["dim"], Lbox=test["Lbox"], r=test["r"]
    )
    try:
      sim.SetInitialConditions(coords)
      if not sim.DetectOverlaps():
        return sim
    except ValueError:
      pass


def compare_move_validation_timing(
  test
) -> dict[str, float]:
  limit = test["Lbox"] * 0.5 - test["r"]
  rng_old = np.random.default_rng(test["seed"])
  rng_new = np.random.default_rng(test["seed"])
  coords_old = _find_valid_coords_old(rng_old, test, limit)
  sim_new = _find_valid_sim_new(rng_new, test, limit)
  coords_new = sim_new.coords
  if coords_new is None:
    raise ValueError("no valid configuration found for new method")

  move_trials = test["move_trials"]
  dl = test["dl"]
  rng_moves = np.random.default_rng(test["seed"])
  indices = rng_moves.integers(0, test["n_spheres"], size=move_trials)
  directions = [
    _random_direction(rng_moves, test["dim"]) for _ in range(move_trials)
  ]

  start_old = time.perf_counter()
  old_accepts = 0
  for idx, direction in zip(indices, directions):
    proposed = coords_old[idx] + dl * direction
    trial_coords = coords_old.copy()
    trial_coords[idx] = proposed
    try:
      validate_initial_conditions(
        trial_coords, test["dim"], test["Lbox"], test["r"]
      )
      old_accepts += 1
    except ValueError:
      pass
  old_time = time.perf_counter() - start_old

  start_new = time.perf_counter()
  new_accepts = 0
  for idx, direction in zip(indices, directions):
    proposed = coords_new[idx] + dl * direction
    if np.any(np.abs(proposed) > limit + 1e-12):
      continue
    if not sim_new._has_overlap_at(proposed, int(idx)):
      new_accepts += 1
  new_time = time.perf_counter() - start_new

  speedup = old_time / new_time if new_time > 0 else float("inf")
  gain_pct = ((old_time - new_time) / old_time * 100.0) if old_time > 0 else 0.0
  return {
    "trials": move_trials,
    "old_accepts": old_accepts,
    "new_accepts": new_accepts,
    "old_time_s": old_time,
    "new_time_s": new_time,
    "speedup": speedup,
    "gain_pct": gain_pct,
  }


test = beta_tests[6]
filename = output_root / test["name"] / "results.npz"
if ReadResultsFromFile and filename.exists():
  data = np.load(filename)
  results = json.loads(str(data["results_json"]))
  params = json.loads(str(data["params_json"]))
else:
  results = compare_move_validation_timing(test)
  params = test
  np.savez(
    filename,
    results_json=json.dumps(results),
    params_json=json.dumps(params),
  )

print(
  "Move validation timing (2D): "
  f"old={results['old_time_s']:.4f}s, new={results['new_time_s']:.4f}s, "
  f"speedup={results['speedup']:.2f}x, gain={results['gain_pct']:.2f}%, "
  f"old accepts={results['old_accepts']}/{results['trials']}, "
  f"new accepts={results['new_accepts']}/{results['trials']}"
)

In [ ]:
# Validation timing test in 2D - one-off test
try:
  # raise Exception("Comment me to do a one-off test")
  test = beta_tests[6]
  results = compare_move_validation_timing(test)
  print(
    "Move validation timing (2D): "
    f"old={results['old_time_s']:.4f}s, new={results['new_time_s']:.4f}s, "
    f"speedup={results['speedup']:.2f}x, gain={results['gain_pct']:.2f}%, "
    f"old accepts={results['old_accepts']}/{results['trials']}, "
    f"new accepts={results['new_accepts']}/{results['trials']}"
  )
except Exception as e:
  print(e)


### validation-timing-3d

Beta test: compare timing for old vs new validation method for $N=17$ in a 3D box with $L=10$ and $r=0.5$.

In [ ]:
# Validation timing test in 3D
test = beta_tests[7]
filename = output_root / test["name"] / "results.npz"
if ReadResultsFromFile and filename.exists():
  data = np.load(filename)
  results = json.loads(str(data["results_json"]))
  params = json.loads(str(data["params_json"]))
else:
  results = compare_move_validation_timing(test)
  params = test
  np.savez(
    filename,
    results_json=json.dumps(results),
    params_json=json.dumps(params),
  )

print(
  "Validation timing (3D): "
  f"old={results['old_time_s']:.4f}s, new={results['new_time_s']:.4f}s, "
  f"speedup={results['speedup']:.2f}x, gain={results['gain_pct']:.2f}%, "
  f"old accepts={results['old_accepts']}/{results['trials']}, "
  f"new accepts={results['new_accepts']}/{results['trials']}"
)

In [ ]:
# Validation timing test in 3D - one-off test
try:
  # raise Exception("Comment me to do a one-off test")
  test = beta_tests[7]
  results = compare_move_validation_timing(test)
  print(
    "Move validation timing (3D): "
    f"old={results['old_time_s']:.4f}s, new={results['new_time_s']:.4f}s, "
    f"speedup={results['speedup']:.2f}x, gain={results['gain_pct']:.2f}%, "
    f"old accepts={results['old_accepts']}/{results['trials']}, "
    f"new accepts={results['new_accepts']}/{results['trials']}"
  )
except Exception as e:
  print(e)
